# Create LTI launch requests to clone git-based repositories

he following steps take place when the user clicks on a link when generates a launch request:

1. A user session is initiated with the LTI launch request (LTI 1.1 or LTI 1.3)
2. The git repository referred to in the link is made up to date in their home directory (keeping local changes if there are merge conflicts)
3. They are shown the specific notebook / directory referred to in the link added to the launch request as a url encoded parameter


## Canvas LMS: Assignment Links vs Custom Fields

The Canvas LMS expects the assignment link to include URL encoded parameters since the request is sent to the External Tool as a POST request (in this case IllumiDesk is the External Tool). Custom Fields, on the other hand, are included with as a string win the `Application Settings` --> `Custom Fields` text box.

To create an output for the Assignment Link, toggle the check box next to the `is_assignment_link` label. If unchecked, the tool will create astring to add to the Custom Field section.

In [1]:
from ipywidgets import interact
from urllib.parse import urlunparse, urlparse, urlencode, parse_qs
from IPython.display import Markdown


@interact
def make_launch_link(is_assignment_link=True, domain_url='https://flatiron.illumidesk.com', repo_url='', branch='master', filepath='', app=['notebook', 'lab']):
    """
    Generate a launch request which clones and merges source files from a git-based
    repository.
    """
    
    # Parse the query to its constituent parts
    scheme, netloc, path, params, query_str, fragment = urlparse(domain_url.strip())
    
    # nbgitpuller takes arguments via query parameters.
    # However, your hub might already require query parameters to function (for example, to pick a profile to launch in)
    # So we preserve the parameters we get & amend them to include the git-pull info we want
    query = parse_qs(query_str, keep_blank_values=True)
    
    
    # Make sure the path doesn't contain multiple slashes
    if not path.endswith('/'):
        path += '/'
    path += 'user-redirect/git-pull'
    assignment_link_path = f'/hub/lti/launch?next={path}'
    path_url = f'next={path}'
    
    # Construct query parameters from 
    for name, val in [('repo', repo_url), ('branch', branch), ('subPath', filepath), ('app', app)]:
        if len(val) > 0:
            query[name] = val.strip().strip('/')
    
    assignment_link_url = urlunparse((scheme, netloc, assignment_link_path, params, urlencode(query, doseq=True), fragment))
    path_url = urlunparse(('', '', path_url, params, urlencode(query, doseq=True), fragment))
    
    if is_assignment_link:
        return assignment_link_url
    return path_url
    
    

interactive(children=(Checkbox(value=True, description='is_assignment_link'), Text(value='https://flatiron.ill…